---

# Atenção!

Lembre-se de clonar este notebook antes de tentar editar as células de código.

Para isso, basta seguir os passos:

File -> Save a copy in Drive



---

# Epílogo

Com as duas últimas aulas, obtivemos uma familiaridade com os problemas do Caixeiro Viajante (TSP) e do Roteamento de Veículos (VRP), que são peças fundamentais em todo este curso. Contudo, ficamos restritos apenas a problemas ou muito pequenos ou artificiais (e.g., com distâncias geradas aleatoriamente), que não representam muito bem a realidade.

Felizmente, este é um dos propósitos do nosso repositório [LoggiBUD](https://github.com/loggi/loggibud). Nesta aula adaptaremos o nosso solver de VRP anterior para lidar com os problemas mais realistas que temos disponíveis.

# VRP no LoggiBUD

## Preliminares

Vamos começar clonando o repositório e instalando suas dependências:

In [7]:
import sys
sys.path.insert(0,'/Users/weslleymoura/workspace/loggibud')

#!git clone https://github.com/loggi/loggibud
#%cd /content/loggibud/

# Instale as dependências do projeto
!pip install poetry
!poetry install
# Se você estiver executando esse script localmente, não precisa dos dois comandos abaixo
#!poetry export -f requirements.txt --without-hashes --output requirements.txt
#!pip install -r requirements.txt

# Verifique se tudo funcionou executando os testes
#!poetry run pytest -s -v tests/

# Baixe os dados compilados
#!wget -nc https://loggibud.s3.amazonaws.com/dataset.zip
#!unzip -n dataset.zip

# Verifique que a pasta `data/` agora não está mais vazia
#!ls data/

Installing dependencies from lock file

No dependencies to install or update

Installing the current project: loggibud (0.1.0)Installing the current project: loggibud (0.1.0)


## Analisando uma instância

Nas duas aulas anteriores, adotamos métodos bem rudimentares e simples para tratar os dados. Por exemplo, cada nó era representado por números de `0` a `n`; as demandas eram dadas por uma lista separada `node_demands`; e a matriz de distâncias já era fornecida.

Em circunstâncias mais práticas pode ser que exista muito mais informações sobre os dados de entrada. Assim, a abordagem anterior de usar uma variável separada para cada informação pode deixar o problema cada vez mais confuso.

No LoggiBUD, nós usamos estruturas bem definidas para os dados conforme descritas no arquivo `loggibud/loggibud/v1/types.py`. Para carregar um dos problemas que temos disponíveis, usamos a `CVRPInstance`.

Vamos exemplificar com um conjunto de dados do Distrito Federal:

In [1]:
from loggibud.v1.types import CVRPInstance

file_path = "../data/cvrp-instances-1.0/train/df-0/cvrp-0-df-0.json"
problem = CVRPInstance.from_file(file_path)
problem

CVRPInstance(name='cvrp-0-df-0', region='df-0', origin=Point(lng=-47.802664728268745, lat=-15.657013854445248), vehicle_capacity=180, deliveries=[Delivery(id='ed0993f8cc70d998342f38ee827176dc', point=Point(lng=-47.7496622016347, lat=-15.65879313293694), size=10), Delivery(id='c7220154adc7a3def8f0b2b8a42677a9', point=Point(lng=-47.75887552060412, lat=-15.651440380492554), size=10), Delivery(id='54e598b20ffa2707d01b1ac00942793f', point=Point(lng=-47.755018806591394, lat=-15.651207309372888), size=7), Delivery(id='64812681ba8c280649f764b9fb40936a', point=Point(lng=-47.758785390289965, lat=-15.648706444367969), size=3), Delivery(id='c17ea5bb3bdb1346ea358516cba81e5c', point=Point(lng=-47.75284167302011, lat=-15.66047286919706), size=10), Delivery(id='fb1b2bc2ac3783706de766e715cb9d01', point=Point(lng=-47.74788198106806, lat=-15.663388754816273), size=3), Delivery(id='c667f779151bd4e23e91d3b4659ffd94', point=Point(lng=-47.75836162961292, lat=-15.648355505737312), size=5), Delivery(id='7b315f

A variável `problem` possui várias propriedades com informações relevantes ao problema. Aqui estão algumas delas:

In [2]:
print(f"A capacidade de cada veículo é: {problem.vehicle_capacity}")
print(f"A localização do ponto de partida é: {problem.origin}")
print(f"Esta instância possui um total de {len(problem.deliveries)} entregas")

A capacidade de cada veículo é: 180
A localização do ponto de partida é: Point(lng=-47.802664728268745, lat=-15.657013854445248)
Esta instância possui um total de 1037 entregas


Observe que a origem aqui é um outro conjunto de dados do tipo `Point`, que contém suas as coordenadas (latitude e longitude).

Neste problema há um total de 1037 entregas. Vamos examinar como elas são fornecidas:

In [3]:
# Vamos analisar a primeira delas. O restante segue o mesmo formato
delivery = problem.deliveries[0]
delivery

Delivery(id='ed0993f8cc70d998342f38ee827176dc', point=Point(lng=-47.7496622016347, lat=-15.65879313293694), size=10)

Assim, cada entrega é do tipo `Delivery`, e elas possuem além das coordenadas como no caso da origem, um identificador e uma demanda representada por `size`.

In [ ]:
# Use este espaço para analisar outras propriedades ou outras entregas deste problema

Finalmente, podemos visualizar todas as entregas em um mapa para termos uma melhor visão do tipo de problema que estamos lidando:

In [4]:
from loggibud.v1.plotting.plot_instance import plot_cvrp_instance


plot_cvrp_instance(problem)
# Experimente mover e dar zoom no mapa
# Se estiver executando este código localmente num shell, complemente com os
# seguintes passos
# plot_cvrp_instance(problem).save_to("map.html")
# Abra o arquivo `map.html` num navegador para ter a mesma experiência daqui

## Calculando a matriz de distâncias do problema

Até o momento possuímos diversas informações para resolver um problema real com os algoritmos desenvolvidos na última aula. Resta apenas determinar as distâncias entre cada nó.

Nas aulas anteriores estas distâncias eram fornecidas. Aqui, precisamos cálculá-las usando as coordenadas de cada nó.

### Distâncias em linha reta

Você provavelmente está familiarizado com a [distância Euclidiana](https://en.wikipedia.org/wiki/Euclidean_distance), que dá a a distância em linha reta entre dois pontos no plano. Como nossos pontos não possuem coordenadas num plano, mas sim na Terra, existe uma versão equivalente chamada [Grande Círculo](https://en.wikipedia.org/wiki/Great_circle) que dá a distância em linha reta entre dois pontos numa esfera.

No nosso repositório já existe uma função própria para isso no módulo `loggibud/loggibud/v1/distances.py` chamada `calculate_distance_matrix_great_circle_m`. Para usá-la, precisamos de uma lista de elementos do tipo `Point`.

No caso, temos a origem `problem.origin` que já é do tipo `Point`, e cada entrega no campo `problem.deliveries` possui dentre suas propriedades um `point`. Logo, basta concatená-las:

In [5]:
points = [problem.origin]
for delivery in problem.deliveries:
    points.append(delivery.point)

# O resultado será uma longa lista de pontos
points

[Point(lng=-47.802664728268745, lat=-15.657013854445248),
 Point(lng=-47.7496622016347, lat=-15.65879313293694),
 Point(lng=-47.75887552060412, lat=-15.651440380492554),
 Point(lng=-47.755018806591394, lat=-15.651207309372888),
 Point(lng=-47.758785390289965, lat=-15.648706444367969),
 Point(lng=-47.75284167302011, lat=-15.66047286919706),
 Point(lng=-47.74788198106806, lat=-15.663388754816273),
 Point(lng=-47.75836162961292, lat=-15.648355505737312),
 Point(lng=-47.71947511765923, lat=-15.696661251647132),
 Point(lng=-47.76340110906013, lat=-15.668974870973102),
 Point(lng=-47.75482460546861, lat=-15.65435446002705),
 Point(lng=-47.64215279414541, lat=-15.878651851861552),
 Point(lng=-47.57874209958461, lat=-15.991660310127441),
 Point(lng=-47.55621211610284, lat=-15.966403613565724),
 Point(lng=-47.60854239379657, lat=-15.891214135632072),
 Point(lng=-47.487542336969646, lat=-15.858580488239232),
 Point(lng=-47.551358952435706, lat=-15.960658771306274),
 Point(lng=-47.36431147883306,

Com isso, a matriz de distâncias pode ser calculada facilmente:

In [6]:
from loggibud.v1.distances import calculate_distance_matrix_great_circle_m


distance_matrix = calculate_distance_matrix_great_circle_m(points)
distance_matrix
# Verifique o tamanho dela com `distance_matrix.shape`

array([[   0.        , 5678.34965138, 4729.31199561, ..., 2577.04094525,
        1731.37569117, 2098.07623024],
       [5678.34965138,    0.        , 1281.23929932, ..., 4165.8722425 ,
        5013.19226651, 4256.121775  ],
       [4729.31199561, 1281.23929932,    0.        , ..., 2930.88117036,
        3838.68919117, 3079.36204452],
       ...,
       [2577.04094525, 4165.8722425 , 2930.88117036, ...,    0.        ,
        1038.87108453,  501.8186876 ],
       [1731.37569117, 5013.19226651, 3838.68919117, ..., 1038.87108453,
           0.        ,  759.95683246],
       [2098.07623024, 4256.121775  , 3079.36204452, ...,  501.8186876 ,
         759.95683246,    0.        ]])

### Distâncias de rua

Enquanto a Grande Círculo pode ser útil em algumas regiões, ela essencialmente assume que nossas entregas são feitas em linha reta (de drones ou helicópteros, por exemplo). Isso acaba por ignorar a geografia da cidade, como a existência de ruas de mão única, bloqueios, pontes etc.

Felizmente, podemos usar um serviço gratuito chamado [Open Source Routing Machine](http://project-osrm.org/) (OSRM) que retorna a distância de rua entre dois pontos. A documentação do repositório ensina como criar um servidor local, mas durante o curso existe um servidor disponível para os alunos.

Como você deve prever, no mesmo módulo `loggibud/loggibud/v1/distances.py` existe uma função `calculate_distance_matrix_m` que calcula a matriz de distâncias usando este servidor. Fora a lista com os pontos como antes, precisamos de uma variável de configurações com o endereço do servidor disponível.

In [ ]:
from loggibud.v1.distances import calculate_distance_matrix_m, OSRMConfig


# Configuração com o servidor para os alunos
osrm_config = OSRMConfig(host="http://ec2-34-222-175-250.us-west-2.compute.amazonaws.com")

# Pode levar alguns segundos
distance_matrix = calculate_distance_matrix_m(points, config=osrm_config)
distance_matrix


## Resolvendo um VRP com uma instância real

Agora temos todos os ingredientes necessários para resolver um problema de roteamento real. Da aula anterior, temos uma função que recebe uma matriz de distâncias, uma lista com as demandas de cada nó, e a capacidade do veículo.

Aqui está esta função tirada do Exercício 4 da aula anterior por conveniência:

In [7]:
from ortools.constraint_solver import pywrapcp


def solve_vrp_ortools2(
    distance_matrix, node_demands, vehicle_capacity
):
    n = distance_matrix.shape[0]  # número de nós do problema
    depot_node = 0  # número do nó que representa o ponto de origem

    # Vamos usar `n` como número de veículos, pois haveria na pior das hipóteses
    # um veículo entregando cada pacote
    num_vehicles = n
    manager = pywrapcp.RoutingIndexManager(n, num_vehicles, depot_node)
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(i, j):
        # `i` e `j` são índices internos do OR-Tools. Precisamos primeiro
        # convertê-los em nós do nosso problema
        ni = manager.IndexToNode(i)
        nj = manager.IndexToNode(j)
        return distance_matrix[ni, nj]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Adiciona a restrição de capacidade
    def demand_callback(from_index):
        """Retorna a demanda de um nó"""
        from_node = manager.IndexToNode(from_index)
        return node_demands[from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback
    )
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        [vehicle_capacity] * num_vehicles,
        True,  # start cumul to zero
        'Capacity'
    )

    # Resolve o problema com métodos default
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    solution = routing.SolveWithParameters(search_parameters)

    # Caso não haja solução factível, retorne uma lista vazia como rotas e o
    # valor -1 como distância total
    if not solution:
        return [], -1

    # Constrói as rotas finais
    def create_vehicle_route(vehicle_index):
        route = []
        index = routing.Start(vehicle_index)
        node = manager.IndexToNode(index)
        route.append(node)

        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            node = manager.IndexToNode(index)
            route.append(node)
        return route

    routes = []
    for vehicle_index in range(num_vehicles):
        # Adicione apenas as rotas com mais que apenas [0, 0], ou seja, apenas
        # aquelas com ao menos três pontos
        route = create_vehicle_route(vehicle_index)
        if len(route) > 2:
            routes.append(route)

    return routes, solution.ObjectiveValue()

Dado um problema do LoggiBUD, podemos usar esta função imediatamente criando uma subfunção agindo como interface. No caso, precisaríamos calcular a matriz de distâncias e uma lista com as demandas de cada nó.

Analise com atenção a função abaixo:

In [29]:
from loggibud.v1.distances import calculate_distance_matrix_m, OSRMConfig


def solve_loggibud_vrp(problem):
    distance_matrix = _compute_distance_matrix(problem)
    node_demands = _compute_node_demands(problem)
    vehicle_capacity = problem.vehicle_capacity

    # Chama o solver de antes
    return solve_vrp_ortools2(distance_matrix, node_demands, vehicle_capacity)

def _compute_distance_matrix(problem):
    osrm_config= OSRMConfig(host="http://localhost:5000")

    points = [problem.origin]
    for delivery in problem.deliveries:
        points.append(delivery.point)

    return calculate_distance_matrix_m(points, config=osrm_config)


def _compute_node_demands(problem):
    """Retorna uma lista com as demandas de cada nó"""
    node_demands = [0]  # inicializa com a demanda nula da origem
    for delivery in problem.deliveries:
        node_demands.append(delivery.size)

    return node_demands

# Vamos experimentar com o problema atual (pode levar um tempo mais longo que
# o de costume já que a instância é maior)
routes, distance = solve_loggibud_vrp(problem)
routes, distance




([[0,
   951,
   942,
   943,
   941,
   954,
   950,
   948,
   944,
   947,
   946,
   24,
   44,
   56,
   22,
   241,
   35,
   40,
   43,
   19,
   54,
   30,
   51,
   37,
   229,
   60,
   34,
   61,
   20,
   42,
   26,
   57,
   495,
   487,
   0],
  [0,
   1016,
   175,
   167,
   181,
   166,
   177,
   188,
   174,
   307,
   800,
   815,
   299,
   292,
   248,
   258,
   260,
   308,
   257,
   270,
   279,
   296,
   266,
   249,
   291,
   267,
   275,
   277,
   774,
   824,
   812,
   833,
   829,
   0],
  [0,
   992,
   76,
   98,
   110,
   111,
   91,
   127,
   93,
   104,
   297,
   303,
   278,
   272,
   276,
   265,
   288,
   298,
   264,
   256,
   263,
   251,
   247,
   268,
   261,
   285,
   293,
   255,
   289,
   301,
   252,
   283,
   250,
   810,
   834,
   0],
  [0,
   837,
   787,
   832,
   818,
   793,
   805,
   105,
   88,
   94,
   123,
   63,
   813,
   826,
   781,
   782,
   798,
   827,
   817,
   791,
   831,
   253,
   300,
   273,
   8

Assumindo que você tenha completado todos os exercícios da aula passada, no Exercício 5 foi desenvolvida uma função para verificar se a solução é factível. Vamos testá-la aqui também:

In [30]:
def evaluate_solution(routes, node_demands, vehicle_capacity):
    """Avalia se a solução de um solver VRP é factível"""

    # O número de nós do problema é o tamanho da lista de demandas
    n = len(node_demands)
    is_feasible1 = _has_all_nodes(routes, n)
    is_feasible2 = _capacities_are_respected(
        routes, node_demands, vehicle_capacity
    )

    # Para a solução ser factível, as duas condições devem ser verdadeiras
    return is_feasible1 & is_feasible2


def _has_all_nodes(routes, n):
    """Verifica se um conjunto de rotas possui todos os elementos
    Se um problema tem, por exemplo, 5 nós, eles são numerados de 0 a 4. Logo,
    no caso geral, devemos coletar todos os nós de cada rota, agrupá-los sem
    repetição, e verificar se todos os nós de 0 a n - 1 estão presentes.
    """
    all_nodes = []
    for route in routes:
        all_nodes += route  # concatena cada rota separadamente

    # Para evitar repetições, vamos transformar a lista `all_nodes` em um
    # conjunto
    all_nodes_set = set(all_nodes)

    # Devemos verificar se este conjunto é igual ao conjunto {0, 1, ..., n - 1}
    expected_nodes_set = set(range(n))

    return all_nodes_set == expected_nodes_set


def _capacities_are_respected(routes, node_demands, vehicle_capacity):
    """Verifica se a demanda total de cada rota não viola a capacidade"""

    def compute_route_total_demand(route, node_demands):
        """Calcula a demanda total de uma rota"""
        total_demand = 0
        for node in route:
            total_demand += node_demands[node]
        return total_demand

    # Itera em cada rota. Se alguma violar a restrição de capacidade, retorne
    # `False`. Se passarmos por todas as rotas sem interrupção, significa que
    # todas respeitaram a restrição, e assim podemos retornar `True`
    for route in routes:
        if compute_route_total_demand(route, node_demands) > vehicle_capacity:
            return False

    return True

node_demands = _compute_node_demands(problem)
evaluate_solution(routes, node_demands, problem.vehicle_capacity)

True

Na minha execução, obtive um `True`, indicando que a solução respeita as restrições de capacidade.

In [ ]:
# Use este espaço para experimentar com outras instâncias
# Lembre que basta trocar a variável
# file_path = "./data/cvrp-instances-1.0/train/df-0/cvrp-0-df-0.json"
# para usar outro arquivo .json e repetir os passos acima

### Aperfeiçoando o retorno

Da mesma forma que temos uma variável do tipo `CVRPInstance` que organiza todos os dados de entrada do problema, temos também uma `CVRPSolution` que organiza a resposta. Atualmente, retornamos uma lista de listas `routes` com os índices de cada nó. Cada rota tem o formato

```python
[0, nós entregues, 0]
```

Um objeto `CVRPSolution` possui os seguintes campos:

```python
class CVRPSolution(JSONDataclassMixin):
    name: str
    vehicles: List[CVRPSolutionVehicle]
```

O `name` é apenas o nome da instância, que pode ser obtido com `problem.name`. O segundo, `vehicles`, é o equivalente de nossas `routes`, e consiste em uma lista de objetos do tipo `CVRPSolutionVehicle`.

Cada `CVRPSolutionVehicle`, por sua vez, tem os campos:

```python
class CVRPSolutionVehicle:
    origin: Point
    deliveries: List[Delivery]
```

em que `origin` é o ponto de partida do problema, obtido de `problem.origin`.
`deliveries`, por sua vez, tem o mesmo tipo de `deliveries` da variável de entrada.

Pode ter sido muita informação de uma só vez, então vamos a um exemplo. Suponhamos que nosso algoritmo retornou as seguintes rotas:

```python
routes = [[0, 1, 2, 0], [0, 3, 4, 5]]
```

Para converter a primeira rota em um `CVRPSolutionVehicle`, podemos fazer algo como:

In [31]:
route1 = [0, 1, 2, 0]
# as entregas serão problem.deliveries[1] e problem.deliveries[2], ou
deliveries = []
for node in route1[1:-1]:
    deliveries.append(problem.deliveries[node - 1])

deliveries

[Delivery(id='ed0993f8cc70d998342f38ee827176dc', point=Point(lng=-47.7496622016347, lat=-15.65879313293694), size=10),
 Delivery(id='c7220154adc7a3def8f0b2b8a42677a9', point=Point(lng=-47.75887552060412, lat=-15.651440380492554), size=10)]

Observe que `route1[1:-1]` ignora o primeiro e o último elementos, que são sempre 0 no nosso caso. Além disso, no nosso código inicial nós juntamos as entregas com a origem, então as entregas começam do índice 1. Por isso precisamos do `node - 1`.

Assim, podemos fazer uma função que retorna uma solução no formato `CVRPSolution` a partir de um conjunto de `routes`:


In [32]:
from loggibud.v1.types import CVRPSolution, CVRPSolutionVehicle


def _create_cvrp_solution(problem, routes):
    vehicles = []
    for route in routes:
        vehicle = _create_cvrp_vehicle(problem, route)
        vehicles.append(vehicle)

    # Com os veículos, construímos o objeto `CVRPSolution`
    return CVRPSolution(
        name=problem.name,
        vehicles=vehicles
    )

def _create_cvrp_vehicle(problem, route):
    """
    Constrói um objeto do tipo `CVRPSolutionVehicle` a partir de uma rota
    """
    deliveries = []
    for node in route[1:-1]:
        deliveries.append(problem.deliveries[node - 1])

    return CVRPSolutionVehicle(origin=problem.origin, deliveries=deliveries)

# Experimente com a nossa solução
_create_cvrp_solution(problem, routes)

CVRPSolution(name='cvrp-0-df-0', vehicles=[CVRPSolutionVehicle(origin=Point(lng=-47.802664728268745, lat=-15.657013854445248), deliveries=[Delivery(id='81fe5978d14a6bf8f07cfaf84cae24e5', point=Point(lng=-47.78106173915415, lat=-15.761658495822878), size=5), Delivery(id='fb7b52ac60dcb52f9030f1fb9d7e2da4', point=Point(lng=-47.78181423302307, lat=-15.76218100059119), size=5), Delivery(id='d061242b7ffb23fbb7f83b23af1358d1', point=Point(lng=-47.782571819442516, lat=-15.764310938439191), size=4), Delivery(id='c298ea1104e4393ff099502bd8e2bdd2', point=Point(lng=-47.78254933836914, lat=-15.765729712399777), size=10), Delivery(id='fec68af4852cc5e372f21adce9571233', point=Point(lng=-47.78282559842971, lat=-15.766917425086774), size=4), Delivery(id='aba9da366cc60836986411f57564eb0b', point=Point(lng=-47.78133101446272, lat=-15.76617503095794), size=8), Delivery(id='ce5ff75f2742b2f55c2f2601ba50eae5', point=Point(lng=-47.78042061289433, lat=-15.765585949617309), size=5), Delivery(id='193bcf61e7fc38b

Observe um detalhe importante: as entregas em cada `CVRPVehicle` estão na ordem em que serão entregues. O algoritmo que usamos internamente acaba executando um TSP para determinar cada rota, então não precisamos nos precupar com isso.

Podemos, então, aperfeiçoar nosso solver anterior para retornar um `CVRPSolution` em vez de uma lista de rotas como antes:


In [34]:
from loggibud.v1.distances import calculate_distance_matrix_m, OSRMConfig
from loggibud.v1.types import CVRPSolution, CVRPSolutionVehicle, CVRPInstance


def solve_loggibud_vrp(problem):
    distance_matrix = _compute_distance_matrix(problem)
    node_demands = _compute_node_demands(problem)
    vehicle_capacity = problem.vehicle_capacity

    # Chama o solver de antes
    routes, distance = solve_vrp_ortools2(
        distance_matrix, node_demands, vehicle_capacity
    )

    # Cria uma solução com o formato `CVRPSolution`
    return _create_cvrp_solution(problem, routes)

def _compute_distance_matrix(problem):
    osrm_config= OSRMConfig(host="http://localhost:5000")

    points = [problem.origin]
    for delivery in problem.deliveries:
        points.append(delivery.point)

    return calculate_distance_matrix_m(points, config=osrm_config)


def _compute_node_demands(problem):
    """Retorna uma lista com as demandas de cada nó"""
    node_demands = [0]  # inicializa com a demanda nula da origem
    for delivery in problem.deliveries:
        node_demands.append(delivery.size)

    return node_demands

def _create_cvrp_solution(problem, routes):
    vehicles = []
    for route in routes:
        vehicle = _create_cvrp_vehicle(problem, route)
        vehicles.append(vehicle)

    # Com os veículos, construímos o objeto `CVRPSolution`
    return CVRPSolution(
        name=problem.name,
        vehicles=vehicles
    )

def _create_cvrp_vehicle(problem, route):
    """
    Constrói um objeto do tipo `CVRPSolutionVehicle` a partir de uma rota
    """
    deliveries = []
    for node in route[1:-1]:
        deliveries.append(problem.deliveries[node - 1])

    return CVRPSolutionVehicle(origin=problem.origin, deliveries=deliveries)

# Vamos experimentar novamente com o problema atual
solution = solve_loggibud_vrp(problem)
solution

CVRPSolution(name='cvrp-0-df-0', vehicles=[CVRPSolutionVehicle(origin=Point(lng=-47.802664728268745, lat=-15.657013854445248), deliveries=[Delivery(id='81fe5978d14a6bf8f07cfaf84cae24e5', point=Point(lng=-47.78106173915415, lat=-15.761658495822878), size=5), Delivery(id='fb7b52ac60dcb52f9030f1fb9d7e2da4', point=Point(lng=-47.78181423302307, lat=-15.76218100059119), size=5), Delivery(id='d061242b7ffb23fbb7f83b23af1358d1', point=Point(lng=-47.782571819442516, lat=-15.764310938439191), size=4), Delivery(id='c298ea1104e4393ff099502bd8e2bdd2', point=Point(lng=-47.78254933836914, lat=-15.765729712399777), size=10), Delivery(id='fec68af4852cc5e372f21adce9571233', point=Point(lng=-47.78282559842971, lat=-15.766917425086774), size=4), Delivery(id='aba9da366cc60836986411f57564eb0b', point=Point(lng=-47.78133101446272, lat=-15.76617503095794), size=8), Delivery(id='ce5ff75f2742b2f55c2f2601ba50eae5', point=Point(lng=-47.78042061289433, lat=-15.765585949617309), size=5), Delivery(id='193bcf61e7fc38b

Estude bem todas estas funções. Se você acompanhou bem os códigos das aulas anteriores, verá que apesar da complexidade estamos construindo funcionalidades de forma gradativa.

Experimente repetir os passos para outras instâncias, e não tenha receio de adicionar novos blocos de código verificando cada variável intermediária.

### Avaliando a solução

Temos agora um tipo de dados mais robusto. Isto é ótimo, mas como avaliaremos se a solução é factível ou não? A princípio você poderia adaptar a função `evaluate_solution` de antes, mas neste caso já existe outra `evaluate_solution` preparada no módulo `loggibud/v1/eval/task1.py`.

Ela recebe o problema e a solução, verifica as restrições operacionais (todos os nós estão contidos nas rotas e as capacidades não são violadas) e acusa um erro caso não sejam satisfeitas. Se tudo correr bem, ela retorna a distância total da rota em km:

In [35]:
from loggibud.v1.distances import OSRMConfig
from loggibud.v1.eval.task1 import evaluate_solution


# Configuração com o servidor para os alunos
osrm_config= OSRMConfig(host="http://localhost:5000")

evaluate_solution(problem, solution, config=osrm_config)

1647.7035

Além disso, podemos visualizar as rotas finais assim como os pontos iniciais:

In [36]:
from loggibud.v1.plotting.plot_solution import plot_cvrp_solution


plot_cvrp_solution(solution)

A função anterior plota cada rota como uma sequência de linhas entre uma entrega e outra. Caso prefira, existe outra função para visualizar as rotas em ruas:

In [37]:
from loggibud.v1.plotting.plot_solution import plot_cvrp_solution_routes


# Observe como ela precisa do servidor do OSRM em funcionamento
plot_cvrp_solution_routes(solution, config=osrm_config)

# Resumo

Nesta aula aplicamos o conhecimento desenvolvido nas duas aulas anteriores a um conjunto de dados mais realista do LoggiBUD. Tratamos de uma instância aqui apenas; existem centenas outros problemas para você experimentar.

O VRP tratado aqui é bastante estudado na literatura, mas ainda não é suficiente para representar os problemas que enfrentamos diariamente em empresas de entregas como a Loggi. Nas próximas aulas veremos como deixá-lo ainda mais realista para nossos propósitos.


# Exercícios

## Exercício 1

**Resolva o VRP usando as seguintes instâncias:**

- `dev/df-0/cvrp-0-df-90.json`
- `dev/pa-0/cvrp-0-pa-90.json`
- `dev/rj-0/cvrp-0-rj-90.json`

Calcule a distância total das soluções e plote as rotas.
O solver foi capaz de resolver todos os problemas?

Sugestão: Interrompa manualmente a execução caso o solver esteja levando muito tempo, como mais de dez minutos.

## Exercício 2

**Use o solver LKH-3 implementado no repositório para resolver as mesmas instâncias do Exercício 1. Compare as distâncias em cada caso.**

Dica: O solver pode ser encontrado em

```python
from loggibud.v1.baselines.task1 import lkh_3
```

e executado simplesmente com

```python
solution = lkh_3.solve(problem, params)
```

em que `params` é um conjunto de parâmetros do solver. Veja novamente a aula 1 se tiver alguma dúvida.

## Exercício suplementar

Experimente pesquisar outros solvers de VRP e, se possível, implemente-os com a mesma interface do solver desta aula. Compare suas soluções nas mesmas instâncias do exercício anterior.

## Tarefa de casa

**Adaptação de código**

Fora a capacidade de entender como uma nova biblioteca ou repositório funciona, a habilidade de adaptar seu código para novas demandas é muito importante para um programador.

Como visto nesta aula, a solução do VRP retornada pelo nosso solver tem o formato:

```python
class CVRPSolution(JSONDataclassMixin):
    name: str
    vehicles: List[CVRPSolutionVehicle]
```

e, dada uma variável `solution` com esta estrutura, podemos fazer algumas operações, como traçar suas rotas e calcular a distância total com a função `evaluate_solution`.

Suponha que, para nossos propósitos, seja interessante uma variável que armazene, fora `name` e `vehicles`, a distância total (em quilômetros) das rotas e o número de veículos usados.

Nesta tarefa você irá adaptar a classe `CVRPSolution` para incluir estes outros parâmetros de interesse. Para isso, você deverá criar uma nova classe, e.g., `CVRPSolution2` (o nome não é relevante, fique à vontade para usar o que quiser), como exemplificado nos blocos de código a seguir.

Resolva a instância `dev/pa-0/cvrp-0-pa-90.json` com o solver desenvolvido aqui, e retorne uma solução como um objeto no novo formato. Ao final, siga o mesmo processo da Etapa 1 para salvar o resultado em um arquivo `solution_loggibud.json`.

In [ ]:
# Use esta célula para resolver a instância desejada e retornar uma variável `solution`

In [ ]:
# Use esta para calcular a distância total e o número de veículos necessários
# total_distance_km = ... (observe que a função `evaluate_solution` já retorna a distância em km)
# num_vehicles = ... (dica: busque pelo número de elementos em `solution.vehicles`)

In [ ]:
from dataclasses import dataclass

from loggibud.v1.types import CVRPSolution


@dataclass
class CVRPSolution2(CVRPSolution):
    total_distance_km: float
    num_vehicles: int

In [ ]:
# Complete os campos a seguir para construir uma nova variável com o novo formato
solution2 = CVRPSolution2(
    name=,
    vehicles=,
    total_distance_km=,
    num_vehicles=,
)

# Salve a instância como na etapa 1 em um arquivo `solution_loggibud.json`